# Examples of the ELO Python Client

## Connect to ELO

In [1]:
from decouple import AutoConfig
from eloservice.elo_service import EloService

def load_elo_service():
    config = AutoConfig(search_path='./') # load .env file from current directory with connection details
    rest_baseurl = config("TEST_ELO_IX_URL")
    rest_user = config("TEST_ELO_IX_USER")
    rest_password = config("TEST_ELO_IX_PASSWORD")
    return EloService(url=rest_baseurl, user=rest_user, password=rest_password)

elo_service = load_elo_service()

## Load some demo dataset from kaggle

In [2]:
# Downloaded from https://www.kaggle.com/datasets/kyanyoga/sample-sales-data/data?select=sales_data_sample.csv
import pandas as pd

#Load Data
df=pd.read_csv('sales_data_sample.csv', encoding='ISO-8859-1')
# print the columns
print(df.columns)

df.head(5)


Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [3]:
# select some columns
df["SALE_AMOUNT"] = df["QUANTITYORDERED"] * df["PRICEEACH"]
df["SALE_PRODUCT"] = df["PRODUCTLINE"]
df["SALE_DATE"] = df["ORDERDATE"]
df["SALE_NAME"] =  df["ORDERNUMBER"].astype(str)+ " " + df["PRODUCTCODE"].astype(str) + " " + df["CUSTOMERNAME"].astype(str)
# remove \n and multiple spaces from sale_name
df["SALE_NAME"] = df["SALE_NAME"].str.replace("\n", " ").str.replace(" +", " ", regex=True)

df = df[["SALE_NAME", "SALE_DATE", "SALE_PRODUCT", "SALE_AMOUNT"]]
df["SALE_DATE"] = pd.to_datetime(df["SALE_DATE"])
df

,SALE_NAME,SALE_DATE,SALE_PRODUCT,SALE_AMOUNT
0,10107 S10_1678 Land of Toys Inc.,2003-02-24,Motorcycles,2871.00
1,10121 S10_1678 Reims Collectables,2003-05-07,Motorcycles,2765.90
2,10134 S10_1678 Lyon Souveniers,2003-07-01,Motorcycles,3884.34
3,10145 S10_1678 Toys4GrownUps.com,2003-08-25,Motorcycles,3746.70
4,10159 S10_1678 Corporate Gift Ideas Co.,2003-10-10,Motorcycles,4900.00
...,...,...,...,...
2818,10350 S72_3212 Euro Shopping Channel,2004-12-02,Ships,2000.00
2819,"10373 S72_3212 Oulu Toy Supplies, Inc.",2005-01-31,Ships,2900.00
2820,10386 S72_3212 Euro Shopping Channel,2005-03-01,Ships,4300.00
2821,10397 S72_3212 Alpha Cognac,2005-03-28,Ships,2116.16


In [9]:
# formatted with yyyyMMddHHmmss
df["SALE_DATE_ELO"]= df["SALE_DATE"].dt.strftime("%Y%m%d%H%M%S")
df

,SALE_NAME,SALE_DATE,SALE_PRODUCT,SALE_AMOUNT,SALE_DATE_ELO
0,10107 S10_1678 Land of Toys Inc.,2003-02-24,Motorcycles,2871.00,20030224000000
1,10121 S10_1678 Reims Collectables,2003-05-07,Motorcycles,2765.90,20030507000000
2,10134 S10_1678 Lyon Souveniers,2003-07-01,Motorcycles,3884.34,20030701000000
3,10145 S10_1678 Toys4GrownUps.com,2003-08-25,Motorcycles,3746.70,20030825000000
4,10159 S10_1678 Corporate Gift Ideas Co.,2003-10-10,Motorcycles,4900.00,20031010000000
...,...,...,...,...,...
2818,10350 S72_3212 Euro Shopping Channel,2004-12-02,Ships,2000.00,20041202000000
2819,"10373 S72_3212 Oulu Toy Supplies, Inc.",2005-01-31,Ships,2900.00,20050131000000
2820,10386 S72_3212 Euro Shopping Channel,2005-03-01,Ships,4300.00,20050301000000
2821,10397 S72_3212 Alpha Cognac,2005-03-28,Ships,2116.16,20050328000000


In [11]:
basePath = "¶Sales¶PythonExampleDataImport"
separator = "¶"

def create_sord_singlethreaded(row):
    folderName = row["SALE_NAME"]
    fullPath = basePath + separator + folderName
    sordID = elo_service.create_folder(fullPath)
    print(f"Created folder {fullPath} with sordID {sordID}")
    elo_service.overwrite_mask_fields(sord_id=sordID,
                                      mask_name="SALE",
                                      metadata={"SALE_NAME": row["SALE_NAME"],
                                                # formatted with yyyyMMddHHmmss
                                                "SALE_DATE":  row["SALE_DATE_ELO"],
                                                "SALE_PRODUCT": row["SALE_PRODUCT"],
                                                "SALE_AMOUNT": row["SALE_AMOUNT"]})
    print(f"Updated metadata for sord {sordID}")
    return sordID

# function that takes a df row and creates a sord with mask 
# and objkeys from the row
def create_sord_multithreaded(row):
    # we need to import and create the elo_service in the function inorder to use it in a multithreaded environment
    from decouple import AutoConfig
    from eloservice.elo_service import EloService
    config = AutoConfig(search_path='./') # load .env file from current directory with connection details
    rest_baseurl = config("TEST_ELO_IX_URL")
    rest_user = config("TEST_ELO_IX_USER")
    rest_password = config("TEST_ELO_IX_PASSWORD")
    elo_service = EloService(url=rest_baseurl, user=rest_user, password=rest_password)
    basePath = "¶Sales¶PythonExampleDataImport"
    separator = "¶"
    folderName = row["SALE_NAME"]
    fullPath = basePath + separator + folderName
    sordID = elo_service.create_folder(fullPath)
    print(f"Created folder {fullPath} with sordID {sordID}")
    elo_service.overwrite_mask_fields(sord_id=sordID, 
                                      mask_name="SALE",
                                      metadata={"SALE_NAME": row["SALE_NAME"],
                                                # formatted with yyyyMMddHHmmss
                                                "SALE_DATE":  row["SALE_DATE_ELO"],
                                                "SALE_PRODUCT": row["SALE_PRODUCT"],
                                                "SALE_AMOUNT": row["SALE_AMOUNT"]})
    print(f"Updated metadata for sord {sordID}")
    return sordID

### Filter some data

In [12]:
# get the first 5 rows for each product
filtered_df = df.groupby("SALE_PRODUCT").head(5)
filtered_df

,SALE_NAME,SALE_DATE,SALE_PRODUCT,SALE_AMOUNT,SALE_DATE_ELO
0,10107 S10_1678 Land of Toys Inc.,2003-02-24,Motorcycles,2871.00,20030224000000
1,10121 S10_1678 Reims Collectables,2003-05-07,Motorcycles,2765.90,20030507000000
2,10134 S10_1678 Lyon Souveniers,2003-07-01,Motorcycles,3884.34,20030701000000
3,10145 S10_1678 Toys4GrownUps.com,2003-08-25,Motorcycles,3746.70,20030825000000
4,10159 S10_1678 Corporate Gift Ideas Co.,2003-10-10,Motorcycles,4900.00,20031010000000
26,10103 S10_1949 Baane Mini Imports,2003-01-29,Classic Cars,2600.00,20030129000000
27,"10112 S10_1949 Volvo Model Replicas, Co",2003-03-24,Classic Cars,2900.00,20030324000000
28,"10126 S10_1949 Corrida Auto Replicas, Ltd",2003-05-28,Classic Cars,3800.00,20030528000000
29,10140 S10_1949 Technics Stores Inc.,2003-07-24,Classic Cars,3700.00,20030724000000
30,"10150 S10_1949 Dragon Souveniers, Ltd.",2003-09-19,Classic Cars,4500.00,20030919000000


In [14]:
## Dump the data to ELO

# iterate over the rows and create a sord for each row
# apply in singlethreaded mode

filtered_df.apply(create_sord_singlethreaded, axis=1)

Created folder ¶Sales¶PythonExampleDataImport¶10107 S10_1678 Land of Toys Inc. with sordID 6331
Updated metadata for sord 6331
Created folder ¶Sales¶PythonExampleDataImport¶10121 S10_1678 Reims Collectables with sordID 6337
Updated metadata for sord 6337
Created folder ¶Sales¶PythonExampleDataImport¶10134 S10_1678 Lyon Souveniers with sordID 6345
Updated metadata for sord 6345
Created folder ¶Sales¶PythonExampleDataImport¶10145 S10_1678 Toys4GrownUps.com with sordID 6351
Updated metadata for sord 6351
Created folder ¶Sales¶PythonExampleDataImport¶10159 S10_1678 Corporate Gift Ideas Co. with sordID 6359
Updated metadata for sord 6359
Created folder ¶Sales¶PythonExampleDataImport¶10103 S10_1949 Baane Mini Imports with sordID 6535
Updated metadata for sord 6535
Created folder ¶Sales¶PythonExampleDataImport¶10112 S10_1949 Volvo Model Replicas, Co with sordID 6543
Updated metadata for sord 6543
Created folder ¶Sales¶PythonExampleDataImport¶10126 S10_1949 Corrida Auto Replicas, Ltd with sord

0       6331
1       6337
2       6345
3       6351
4       6359
26      6535
27      6543
28      6555
29      6563
30      6570
212     6390
213     6398
214     6406
215     6414
216     6422
475     6557
476     6565
477     6573
478     6581
479     6589
552     6969
553     6977
554     6984
555     6991
556     6996
935     6520
936     6528
937     6536
938     6544
939     6553
1065    6729
1066    6737
1067    6745
1068    6753
1069    6761
dtype: int64

In [15]:
from pandarallel import pandarallel

## Lets try with more data
filtered_df = df.groupby("SALE_PRODUCT").head(100)
pandarallel.initialize(progress_bar=True)
filtered_df.parallel_apply(create_sord_multithreaded, axis=1)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


0       6331
1       6337
2       6345
3       6351
4       6359
        ... 
2526    6948
2527    6953
2528    6959
2529    6967
2530    6975
Length: 677, dtype: int64

In [16]:
# Lets try with all data
pandarallel.initialize(progress_bar=True, nb_workers=12)
df.parallel_apply(create_sord_multithreaded, axis=1)


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


0       6331
1       6337
2       6345
3       6351
4       6359
        ... 
2818    9135
2819    9138
2820    9142
2821    9143
2822    9145
Length: 2823, dtype: int64